# 🔧 Feature Engineering for CSK Match Prediction

This notebook focuses on creating advanced features for CSK match outcome prediction.

## 📋 Contents
1. **Data Loading** - Load processed data from ETL pipeline
2. **Basic Features** - Create fundamental match features
3. **Performance Features** - Batting, bowling, team performance metrics
4. **Historical Features** - Rolling statistics and trends
5. **Contextual Features** - Match importance, opponent strength
6. **Interaction Features** - Feature combinations
7. **Feature Selection** - Select best features for modeling
8. **Feature Validation** - Quality checks and validation

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.append('../src')

from features.build_features import CSKFeatureEngineer
from data.load import DataLoader

# Setup
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

print("🔧 Feature Engineering Notebook Ready!")

## 📥 1. Data Loading

In [ ]:
# Load processed data
loader = DataLoader('../data')
df = loader.load_processed_data('csk_match_level_data.csv')

print(f"📊 Data Shape: {df.shape}")
print(f"📅 Date Range: {df['date'].min()} to {df['date'].max()}")
print(f"🏏 CSK Win Rate: {df['csk_won_match'].mean():.2%}")

df.head()

## 🏗️ 2. Feature Engineering Pipeline

In [ ]:
# Initialize feature engineer
engineer = CSKFeatureEngineer()

# Create all features
df_features = engineer.create_all_features(df)

print(f"✅ Features created! New shape: {df_features.shape}")
print(f"📈 Features added: {df_features.shape[1] - df.shape[1]}")

# Show new columns
new_columns = set(df_features.columns) - set(df.columns)
print(f"\n🆕 New Features ({len(new_columns)}):")
for col in sorted(new_columns):
    print(f"  - {col}")

## 📊 3. Feature Analysis

In [ ]:
# Analyze feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

# Key features to analyze
key_features = [
    'csk_won_toss', 'is_home_venue', 'is_peak_season',
    'strong_opponent', 'is_playoff', 'season_experience'
]

for i, feature in enumerate(key_features):
    if feature in df_features.columns:
        if df_features[feature].dtype in ['int64', 'float64']:
            df_features[feature].hist(bins=20, ax=axes[i], alpha=0.7)
        else:
            df_features[feature].value_counts().plot(kind='bar', ax=axes[i])
        
        axes[i].set_title(f'{feature}')
        axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🎯 4. Feature Importance Analysis

In [ ]:
# Correlation with target
numerical_features = df_features.select_dtypes(include=[np.number]).columns
correlations = df_features[numerical_features].corrwith(df_features['csk_won_match']).abs().sort_values(ascending=False)

# Plot top correlations
plt.figure(figsize=(10, 8))
correlations.head(15).plot(kind='barh')
plt.title('Top 15 Features by Correlation with Match Outcome')
plt.xlabel('Absolute Correlation')
plt.tight_layout()
plt.show()

print("🔝 Top 10 Features by Correlation:")
for i, (feature, corr) in enumerate(correlations.head(10).items(), 1):
    print(f"{i:2d}. {feature:<25} {corr:.4f}")

## 🔍 5. Feature Selection

In [ ]:
# Prepare features for selection
X = df_features.drop(columns=['csk_won_match'])
y = df_features['csk_won_match']

# Select best features
X_selected, selected_features = engineer.select_best_features(X, y, k=20)

print(f"✅ Selected {len(selected_features)} best features:")
for i, feature in enumerate(selected_features, 1):
    print(f"{i:2d}. {feature}")

# Save selected features
feature_info = {
    'selected_features': selected_features,
    'feature_count': len(selected_features),
    'selection_method': 'SelectKBest with f_classif'
}

print(f"\n📊 Final feature set ready for modeling: {X_selected.shape}")

## 💾 6. Save Engineered Features

In [ ]:
# Save feature-engineered dataset
loader = DataLoader('../data')
saved_path = loader.save_feature_engineered_data(df_features, feature_info)

print(f"✅ Feature-engineered data saved to: {saved_path}")
print(f"📊 Final dataset shape: {df_features.shape}")
print(f"🎯 Ready for model training!")

## 📋 Summary

### ✅ Completed Tasks:
1. **Loaded processed data** from ETL pipeline
2. **Created comprehensive features** including:
   - Match context features (toss, venue, season)
   - Performance features (historical stats)
   - Contextual features (opponent strength, match importance)
   - Interaction features (feature combinations)
3. **Analyzed feature importance** and correlations
4. **Selected best features** for modeling
5. **Saved engineered dataset** for ML pipeline

### 🎯 Next Steps:
- Proceed to model training notebook
- Use engineered features for ML pipeline
- Evaluate feature importance in trained models